# Задание : написать программу, которая составляет рейтинг фильмов на основе оценок пользователей сайта IMDb.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ast import literal_eval
#Библиотека Ast и пакет Literal_eval пригодятся для извлечения информации из «сырых» форматов данных

In [3]:
import pandas as pd

In [4]:
path_to_dataset = 'movies_metadata_fixed.csv'
dataset = pd.read_csv(path_to_dataset)
dataset.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92


In [5]:
dataset['genres'].iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [6]:
literal_eval(dataset['genres'].iloc[0])
#На выходе Literal_eval всё тот же список словарей, но теперь он не в виде строки, а в виде объектов языка Python.

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [7]:
[i['name'] for i in literal_eval(dataset['genres'].iloc[0])]

['Animation', 'Comedy', 'Family']

Теперь преобразуем поле genres целиком так же, как и в примере выше

In [8]:
dataset['genres'] = dataset['genres'].fillna('[]')\
                                     .apply(literal_eval)\
                                     .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [9]:
dataset['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

Получим полный список жанров

In [10]:
import functools
available_genres = functools.reduce(lambda x,y: set(x).union(set(y)), dataset.genres.tolist())
available_genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

In [11]:
dataset['year'] = pd.to_datetime(dataset['release_date'])

In [12]:
def process_dataset(df): 
    # Удаление фильмов, для которых отсутствует информация о среднем рейтинге и кол-ве проголосовавших пользователей. 
    rank_dataset = dataset.query('vote_average>0 & vote_count>0')
    
    # Оставим в нашем датасете только колонки: 'title', 'year', 'vote_count', 'vote_average', 'genres'
    usecols = ['title', 'year', 'vote_count', 'vote_average', 'genres']
    rank_dataset = rank_dataset[usecols]
    
    return rank_dataset

In [13]:
dataset = process_dataset(dataset.copy())

In [14]:
dataset.tail()

,title,year,vote_count,vote_average,genres
45456,Caged Heat 3000,1995-01-01,1,3.5,[Science Fiction]
45457,Robin Hood,1991-05-13,26,5.7,"[Drama, Action, Romance]"
45458,Subdue,NaT,1,4.0,"[Drama, Family]"
45459,Century of Birthing,2011-11-17,3,9.0,[Drama]
45460,Betrayal,2003-08-01,6,3.8,"[Action, Drama, Thriller]"


In [15]:
dataset['vote_average']

0        7.7
1        6.9
2        6.5
3        6.1
4        5.7
        ... 
45456    3.5
45457    5.7
45458    4.0
45459    9.0
45460    3.8
Name: vote_average, Length: 42465, dtype: float64

In [16]:
#sort = dataset.sort_values('vote_average', ascending=False)
#sort.head(5)

In [17]:
def recommend_simple(input_films, n_items):
    rank = input_films.copy()
    rank.sort_values('vote_average', ascending=False)
    return rank.head(n_items).reset_index(drop=True)

recommend_simple(dataset, 10)

,title,year,vote_count,vote_average,genres
0,Toy Story,1995-10-30,5415,7.7,"[Animation, Comedy, Family]"
1,Jumanji,1995-12-15,2413,6.9,"[Adventure, Fantasy, Family]"
2,Grumpier Old Men,1995-12-22,92,6.5,"[Romance, Comedy]"
3,Waiting to Exhale,1995-12-22,34,6.1,"[Comedy, Drama, Romance]"
4,Father of the Bride Part II,1995-02-10,173,5.7,[Comedy]
5,Heat,1995-12-15,1886,7.7,"[Action, Crime, Drama, Thriller]"
6,Sabrina,1995-12-15,141,6.2,"[Comedy, Romance]"
7,Tom and Huck,1995-12-22,45,5.4,"[Action, Adventure, Drama, Family]"
8,Sudden Death,1995-12-22,174,5.5,"[Action, Adventure, Thriller]"
9,GoldenEye,1995-11-16,1194,6.6,"[Adventure, Action, Thriller]"


Мы получили список фильмов с самым высоким рейтингом, но легко заметить, что этот рейтинг был сформирован из оценок только одного пользователя. Думаю, ты бы не стал смотреть ничего из этого списка. Как справится с этой незадачей? Простой способ: учесть количество средних оценок для каждого фильма, возвращая их произведение, это самый простой способ учесть количество

In [18]:
def better_formula(x):
    v = x.vote_count#извлекаем количество оценивших vote_count
    R = x.vote_average#извлекаем среднюю оценку vote_average
    return v*R #возвращаем их произведение, это самый простой способ учесть количество

def recommend_better(input_films, n_items):
    rank = input_films.copy()
    rank['formula'] = better_formula(rank)#применим формулу ко всем строкам при помощи лямбда-функции
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_better(dataset, 10)

,title,year,vote_count,vote_average,genres,formula
0,Inception,2010-07-14,14075,8.1,"[Action, Thriller, Science Fiction, Mystery, A...",114007.5
1,The Dark Knight,2008-07-16,12269,8.3,"[Drama, Action, Crime, Thriller]",101832.7
2,Interstellar,2014-11-05,11187,8.1,"[Adventure, Drama, Science Fiction]",90614.7
3,The Avengers,2012-04-25,12000,7.4,"[Science Fiction, Action, Adventure]",88800.0
4,Avatar,2009-12-10,12114,7.2,"[Action, Adventure, Fantasy, Science Fiction]",87220.8
5,Deadpool,2016-02-09,11444,7.4,"[Action, Adventure, Comedy]",84685.6
6,Fight Club,1999-10-15,9678,8.3,[Drama],80327.4
7,Django Unchained,2012-12-25,10297,7.8,"[Drama, Western]",80316.6
8,Guardians of the Galaxy,2014-07-30,10014,7.9,"[Action, Science Fiction, Adventure]",79110.6
9,Pulp Fiction,1994-09-10,8670,8.3,"[Thriller, Crime]",71961.0


Можно ли сделать лучше? Да.

Наша задача - порекомендовать один или несколько фильмов, у которых по-настоящему высокий рейтинг, даже если сейчас его оценили всего несколько человек. Каким был бы рейтинг такого фильма, если бы его посмотрело больше людей? Мы можем только предполагать, поэтому нам остается лишь вариант со средним арифметическим, потому что по статистике, это будет наилучшая оценка среднего.

Если есть отзывы о фильме и какая-то начальная средняя оценка (даже состоящая из 1-го голоса), нечестно её не учитывать.

Для нашей финальной формулы мы попробуем учесть и среднее арифметическое, и текущую оценку одновременно. Но как? Ответ: взвешенное среднее. Это такое среднее, которое учитывает важность отдельных элементов в формуле. В нашем случае, чем больше людей (vote_count) оставили оценку фильму, тем важнее эта информация для общего рейтинга.

In [19]:
def the_best_formula(x, Q, C):
    v = float(x['vote_count'])
    R = x['vote_average']
    return (v/(v+Q) * R) + (Q/(Q+v) * C)
    #измеряем взвешенное среднее, в случае большого количества оценок 'v'
    #будет большой вес (v/(v+Q) у'vote_average'
    #при мало числе оценок 'v', больший вес будет у средней оценки всех фильмов

def recommend_best(input_films, n_items):
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis=1)
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_best(dataset, 10)

Minimum number of votes: 475. Average overall rating: 6.014859295890418


,title,year,vote_count,vote_average,genres,formula
0,The Shawshank Redemption,1994-09-23,8358,8.5,"[Drama, Crime]",8.366360
1,The Godfather,1972-03-14,6024,8.5,"[Drama, Crime]",8.318366
2,The Dark Knight,2008-07-16,12269,8.3,"[Drama, Action, Crime, Thriller]",8.214827
3,Fight Club,1999-10-15,9678,8.3,[Drama],8.193092
4,Pulp Fiction,1994-09-10,8670,8.3,"[Thriller, Crime]",8.181308
5,Forrest Gump,1994-07-06,8147,8.2,"[Comedy, Drama, Romance]",8.079617
6,Schindler's List,1993-11-29,4436,8.3,"[Drama, History, War]",8.078977
7,Whiplash,2014-10-10,4376,8.3,[Drama],8.076244
8,Spirited Away,2001-07-20,3968,8.3,"[Fantasy, Adventure, Animation, Family]",8.055696
9,The Empire Strikes Back,1980-05-17,5998,8.2,"[Adventure, Action, Science Fiction]",8.039651


# Итоги

Итоги: разработал простую рекомендательную систему. Она строится на предположении, что фильмы, которые нравятся большинству, понравятся и тебе. Конечно, это не совсем так, поэтому в следующих заданиях мы учтём персональные вкусы и запрограммируем более интеллектуальные рекомендательные системы.